In [1]:
import numpy as np
import pickle
import pandas as pd

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os

BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"

In [5]:
path = os.path.join(BYU_UW_root, 'networks', 'expandable_octahedron_net.pkl')
net = Network_custom().load_network(path)
net.l_scalar

0.9998951102800386

In [2]:
import numpy as np

def generate_expandable_octahedron(scale=1.0, q_cable=1.0, q_strut=-1.0):
    """
    Generate vertices, edges, force densities, and fixed vertices for an expandable octahedron tensegrity structure.
    
    Parameters:
    scale   : float - Scaling factor for output coordinates
    q_cable : float - Force density for cables (positive)
    q_strut : float - Force density for struts (negative)
    
    Returns:
    vertices       : np.array - Array of vertex coordinates
    edges         : list    - List of edges connecting vertex indices
    force_densities : list - List of force densities for each edge
    fixed         : list    - List of fixed vertex indices
    """
    # Define vertex coordinates 
    vertices = np.array([
        [-0.5, 0, -1],  # 1 
        [-0.5, 0, 1],  # 2
        [0.5, 0, -1],   # 3  #old: 
        [0.5, 0, 1],   # 4 
        [0, 1, -0.5],  # 5
        [0, -1, -0.5],    # 6  #old: 
        [-1, 0.5, 0],  # 7
        [1, 0.5, 0],   # 8
        [-1, -0.5, 0],  # 9
        [1, -0.5, 0],   # 10
        [0, 1, 0.5],  # 11
        [0, -1, 0.5]    # 12
        ]) * scale
    
    # Define edges
    # edges_cables = [[1, 7], [1, 9], [5, 1], [1, 6], [3,5], [6, 3], [3, 8], [3, 10], [5, 7], [5, 8], [6,9], [6, 10], [2, 7], [7, 11], [9, 2], [9, 12], 
            #  [8, 11], [4, 8], [10, 12], [4, 10], [2, 11], [11, 4], [12, 2], [4,12]
    # ]
    edges_cables = [[7, 1], [1, 9], [1, 5], [1, 6], [3,5], [3,6], [3, 8], [3, 10], [5, 7], [5, 8], [6,9], [6, 10], [2, 7], [7, 11], [2, 9], [9, 12], 
             [8, 11], [4, 8], [10, 12], [4, 10], [2, 11], [4, 11], [2, 12], [4,12]
    ]
    edges_struts = [[1,2], [3,4], [5, 6], [7, 8], [9, 10], [11, 12]]
    edges = edges_cables + edges_struts
    
    edges = [[i-1, j-1] for i, j in edges]
    cable_slice = slice(0, len(edges_cables))
    strut_slice = slice(len(edges_cables), len(edges_cables) + len(edges_struts))
    # Assign force densities
    force_densities = [q_cable] * len(edges_cables) + [q_strut] * len(edges_struts)
    
    # Fix the bottom vertex to prevent free movement
    fixed = [0, 2, 5]#[1, 3, 6]
    # fixed = [0, 1, 4]#[1, 3, 6]
    
    return vertices, edges, force_densities, fixed, cable_slice, strut_slice
model_name = 'expandable_octahedron'
# Example usage
q_scalar = 0.012
vertices, edges, force_densities, fixed, cable_slice, strut_slice = generate_expandable_octahedron(scale=40, q_cable=1*q_scalar, q_strut=-1.5*q_scalar)

paths = [[3, 5, 4, 2, 1, 14, 20,21,23,22], [0, 13, 16,6, 7, 18,15], [10,11,19,17,9,8,12]]
directions = np.ones(len(edges))

# net = Network_custom.from_fd(vertices, edges, force_densities, fixed, paths = None, dir = None)
net = Network_custom.direct(vertices, edges, force_densities, fixed, paths = paths, dir = directions)
net.net_plot(color = True,vlabels = True, elables=False, path_colors=True)

np.max(net.f /0.078294515)

7.508540518680814

In [20]:
net.q, net.l1[-1]/net.l1[0]

(array([ 0.012,  0.012,  0.012,  0.012,  0.012,  0.012,  0.012,  0.012,
         0.012,  0.012,  0.012,  0.012,  0.012,  0.012,  0.012,  0.012,
         0.012,  0.012,  0.012,  0.012,  0.012,  0.012,  0.012,  0.012,
        -0.018, -0.018, -0.018, -0.018, -0.018, -0.018]),
 1.632993161855452)

In [14]:
import numpy as np

def unwrap_point_cloud(points, axis):
    """
    Unwraps a cloud of 3D points around a given vector.
    
    Parameters:
    points : (N,3) array - 3D points
    axis   : (3,) array - The axis vector
    
    Returns:
    unwrapped_points : (N,2) array - Unwrapped (θ, z) coordinates
    """
    # Normalize axis
    axis = axis / np.linalg.norm(axis)

    # Compute projections of points onto the axis
    z = np.dot(points, axis)

    # Get radial vectors (orthogonal component to axis)
    radial_vectors = points - np.outer(z, axis)

    # Compute angles θ (atan2 gives full 360° angle)
    theta = np.arctan2(radial_vectors[:,1], radial_vectors[:,0])

    # Compute radial distances
    r = np.linalg.norm(radial_vectors, axis=1)

    # Unwrap by converting θ to linear distance
    unwrapped_x = theta * np.mean(r)  # Scale angle into a length

    return np.column_stack((unwrapped_x, z))

# Example usage
axis = np.array([0, 0, 1])  # Example: Unwrapping around Z-axis
vertices_unwrapped = unwrap_point_cloud(net.vertices, axis)


In [15]:
net.vertices_2d = vertices_unwrapped
net.edges = np.delete(net.edges, strut_slice, axis=0)
net.q = np.delete(net.q, strut_slice, axis=0)
net.l1 = np.delete(net.l1, strut_slice, axis=0)

net.net_plot(plot_type = 'projection', color = True,vlabels = True, elables=False)
rotate_angle = -90
font_scale = 3
tikz_text = net.tikz_string(net.vertices_2d, net.edges, labels = True, rotate_label = rotate_angle, font_scale = font_scale)
with open("tikz_tensegrity0.tex", "w") as f:
    f.write("\n".join(tikz_text))

In [16]:
# TPUc = {'E':130, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU conductive'} # Conductive TPU. Manufacturer Ninjatek Eel
# TPU = {'E':77, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU non-conductive'} # Conductive TPU. Manufacturer Ninjatek Eel

file_path = os.path.join(BYU_UW_root, 'Avg_Stress_Strain_Overture_TPU.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path)

TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive

# net.set_material(TPU) # By saving the material properties in the network, we can easily see how the network was constructed.

# If all elements will get the same material, you can use the following line
# E = [TPU['E']]*len(net.edges)
A = [TPU_nl['A']]*len(net.edges)
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
# l0, l_scalar = net.materialize(E, A)
# Yielding the initial lengths of each element. And the following scalar: min(l0/l1)

net.l0, net.l_scalar

(array([40.65745696, 40.65745696, 40.65745696, 40.65745696, 40.65745696,
        40.65745696, 40.65745696, 40.65745696, 40.65745696, 40.65745696,
        40.65745696, 40.65745696, 40.65745696, 40.65745696, 40.65745696,
        40.65745696, 40.65745696, 40.65745696, 40.65745696, 40.65745696,
        40.65745696, 40.65745696, 40.65745696, 40.65745696]),
 0.829916864955115)

In [12]:
copied_vertices = net.account_for_crossings()
net.net_plot(plot_type = 'projection', color = True,vlabels = True, elables=False)
net._set_leafs()

tikz_text = net.tikz_string(net.vertices_2d, net.edges, labels = True, rotate_label = rotate_angle, font_scale = font_scale)
with open("tikz_tensegrity1.tex", "w") as f:
    f.write("\n".join(tikz_text))

net.l0[net.leaf_edges] -= 2
copied_vertices, net.l0

([(8, 12), (0, 13), (1, 14)],
 array([40.65745696, 40.65745696, 40.65745696, 38.65745696, 40.65745696,
        40.65745696, 40.65745696, 40.65745696, 40.65745696, 40.65745696,
        40.65745696, 40.65745696, 40.65745696, 40.65745696, 40.65745696,
        40.65745696, 40.65745696, 40.65745696, 40.65745696, 40.65745696,
        40.65745696, 40.65745696, 38.65745696, 40.65745696]))

In [13]:
# net.initialize_shape_optimizer(function_type = 'no optimization',  method = 'L-BFGS-B', params = None)
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'L-BFGS-B',options ={"maxiter": 10000, "maxfun": 1500000, "maxls": 5000})
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 10000, "damping": .1, "correction_scalar": 1., "tol": 1e-6})
# net.initialize_shape_optimizer(function_type = 'standard',  method = 'trust-constr')
# net.initialize_shape_optimizer(function_type = 'sigmoid',  method = 'L-BFGS-B', params = {'a': 4, 'b': -1})
net.optimize_vertices()
net.net_plot(color=True, plot_type='optimized', elables=False, vlabels = True)

tikz_text = net.tikz_string(net.vertices_optimized[...,:2], net.edges, labels = True, rotate_label = rotate_angle, font_scale = font_scale)
with open("tikz_tensegrity2.tex", "w") as f:
    f.write("\n".join(tikz_text))

Iteration 0: Current error = 2180.628811066101
Iteration 100: Current error = 95.21114691812848
Iteration 200: Current error = 12.973960196292131
Iteration 300: Current error = 3.4562083767687044
Iteration 400: Current error = 1.5447585736086105
Iteration 500: Current error = 0.8199842802533871
Iteration 600: Current error = 0.44916650013930387
Iteration 700: Current error = 0.24642444671640176
Iteration 800: Current error = 0.13494855187133045
Iteration 900: Current error = 0.0737963716183028
Iteration 1000: Current error = 0.040325083748488415
Iteration 1100: Current error = 0.0220298562661962
Iteration 1200: Current error = 0.012029089092858656
Iteration 1300: Current error = 0.006569787186876826
Iteration 1400: Current error = 0.003587900880434616
Iteration 1500: Current error = 0.001959050625209735
Iteration 1600: Current error = 0.0010696578722087697
Iteration 1700: Current error = 0.0005841433197034937
Iteration 1800: Current error = 0.0003190187711294134
Iteration 1900: Current

In [ ]:
%matplotlib qt
import matplotlib.animation as animation
ani = net.ShapeOptimizer.animate_optimization()
writer = animation.FFMpegWriter(fps=15)
ani.save(os.path.join(BYU_UW_root, 'images', 'form finding animations', f'form_finding_{model_name}_{0}.mp4'), writer=writer)

In [ ]:
edges_to_flip = [5,2,14,21,22,0,16,6,15,10,19,9,12]
for edge_i in edges_to_flip:
    net.edges[edge_i] = [net.edges[edge_i][1], net.edges[edge_i][0]]
for path in net.path:
    for edge in path:
        print(edge, [net.edges[edge][0]+1, net.edges[edge][1]+1])

In [ ]:
reference_point = [0,0,0]                         # The network will be scaler relative to this point
net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically

# #  You can plot the scaled network like this
# # net.net_plot(color=True, plot_type='scaled')

# # Determine the Radius and Angle of the circle that define the arc of each element
R, th = net.arc_param()
# # for the arc length (net.l1 * net.l_scalar) is used unless specified otherwise, for the cord length (net.l0) is used unless specified otherwise
xyz = net.arc_points(n = 10) 

# net.net_plot(color=True, plot_type='arcs', elables=True, vlabels = True)

In [ ]:
# net.flip_curve(2, n = 10)          # Flip the curvature of one edge
net.auto_flip_curves(n = 10)        # Automatically flip the curvature of the edges. Directions will become 1, -1, 1, -1, ...
# net.flip_curves()                   # Flip all the edges.
# net.flip_curves([2, 5, 8], n = 10)       # Flip the curvature of multiple edges
net.net_plot(color=True, plot_type='arcs', elables=True)

In [ ]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
# When no interpolation function is provided, linear interpolation is used. No other interpolation function is implemented (yet?).
# net.jump_at_intersection(intersection_width = 2, intersection_height = 5, interpolation_function=None) # These settings are just for visibility, this would be way to much
# These setting are more appropriate for printing
net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 
net.net_plot(color=True, plot_type='arcs', elables=True)

In [ ]:
alpha_loop = np.deg2rad(30) # The angle of the loop
L_loop  = 3.5                # The length of the loop
n_points = 60               # The number of points in the loop

start_loop_bools = [True, False, False]
end_loop_bools = [True, False, False]

net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)

L_real = L_loop + L_loop*np.tan(alpha_loop)
L_real

In [ ]:
start_loop_bools = [False, True, True]
end_loop_bools = [False, True, False]
L_loop  = 1.5
n_points = 5
net.add_running_start(start_loop_bools, end_loop_bools, L_loop, n_points)

In [ ]:
net.save_network(os.path.join(BYU_UW_root, 'networks', model_name + '_net.pkl'))

In [ ]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
start_gcode     = open(r'DATA/start_gcode.gcode', 'r').read()
end_gcode       = open(r'DATA/end_gcode.gcode', 'r').read()

temperature_settings = {'first_layer_bed_temperature': 65, 'first_layer_temperature':205, 'K-factor': 0.20}
g_code = replace_brackets(start_gcode, temperature_settings)

comment = ''

bed_width = 230
bed_height = 210

gen = G_code_generator(printing_params=printing_params)

In [ ]:
point0, point1 = [20,15,0.3], [90,15,0.3]
point2, point3 = [90,10,0.3], [20,10,0.3]
g_code += gen.move_to_point(point0[0:2], point0[2] + printing_params['nozzle_lift'], comment='Move to start point')
g_code += gen.move_to_point(point0[0:2], point0[2], comment='Lower Nozzle')
g_code += gen.unretract()
g_code += gen._print_line(
        point0=point0,
        point1=point1,
        move_to_start=False, # move to start point without extruding
        extrude_factor=printing_params['extrude_factor']*2.5,
        comment=comment)
g_code += gen.retract()
g_code += gen.move_to_point(point2[0:2], point2[2] + printing_params['nozzle_lift'], comment='Move to start point')
g_code += gen.move_to_point(point2[0:2], point2[2], comment='Lower Nozzle')
g_code += gen.unretract()
g_code += gen._print_line(
        point0=point2,
        point1=point3,
        move_to_start=False, # move to start point without extruding
        extrude_factor=printing_params['extrude_factor']*2.5,
        comment=comment)
g_code += gen.retract()
g_code += gen.wipe(2 * np.pi) # Wipe the nozzle horizontally

In [ ]:
for path_i, cor_list in enumerate(net.paths_xyz):
    g_code += '\n;Path (' + str(path_i) + ' ' + str(path_i) + ')\n'
    cor_list   = np.array(cor_list)
    # Move the coordinates to the center of the bed and add the layer height
    cor_list[:,0] += bed_width/2  - 30
    cor_list[:,1] += bed_height/2 - 50
    cor_list[:,2] += printing_params['layer_height']
    g_code += '\n'
    # Move the coordinates to the start point
    g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2] + printing_params['nozzle_lift'], comment='Move to start point')
    # Lower the nozzle
    g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2], comment='Lower Nozzle')
    # Unretract the filament
    g_code += gen.unretract()
    # Print the path
    for point0, point1 in zip(cor_list[:-1], cor_list[1:]):
        g_code += gen._print_line(
            point0=point0,
            point1=point1,
            move_to_start=False, # move to start point without extruding
            extrude_factor = printing_params['extrude_factor'],
            comment=comment)
    # Retract the filament
    g_code += gen.retract()
    # Wipe the nozzle
    g_code += gen.wipe_from_last_points(g_code)    
    # Raise the nozzle
    g_code += gen.move_to_point(point1[0:2], point1[2] + printing_params['nozzle_lift'], speed_factor=0.5, comment='Raise Nozzle')

g_code += end_gcode
with open(os.path.join('DATA', 'generated_gcodes', model_name + '.gcode'), "w") as g_code_file:
    g_code_file.write(g_code)
print('G-code generated')

In [ ]:
minx, miny = 90, 90
maxx, maxy = 0, 0
for path_i, cor_list in enumerate(net.paths_xyz):
    cor_list   = np.array(cor_list)
    cor_list[:,0] += bed_width/2 -30
    cor_list[:,1] += bed_height/2 -50
    minx = min(minx, np.min(cor_list[:,0]))
    miny = min(miny, np.min(cor_list[:,1]))
    maxx = max(maxx, np.max(cor_list[:,0]))
    maxy = max(maxy, np.max(cor_list[:,1]))

minx , miny , maxx , maxy 

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot vertices
# vertices = np.concatenate((vertices_unwrapped[cable_slice], np.ones((len(vertices_unwrapped[cable_slice]), 1))), axis=1)
# vertices = np.concatenate((net.vertices_optimized, np.ones((len(net.vertices_optimized), 1))), axis=1)

# ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], c='r', marker='o')

# Plot edges
l0 = []
for edge in net.edges[cable_slice]:
    v1, v2 = vertices[edge[0]], vertices[edge[1]]
    l0.append(np.linalg.norm(v1-v2))
    ax.plot([v1[0], v2[0]], [v1[1], v2[1]], [v1[2], v2[2]], 'b-')
    ax.plot(v1[0], v1[1], v1[2], 'b*')
for edge in edges[strut_slice]:
    v1, v2 = vertices[edge[0]], vertices[edge[1]]
    l0.append(np.linalg.norm(v1-v2))
    ax.plot([v1[0], v2[0]], [v1[1], v2[1]], [v1[2], v2[2]], 'r-')
for i, v in enumerate(vertices):
    ax.text(v[0], v[1], v[2], str(i+1), color='blue')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
# ax.set_xlim(-15, 15)
# ax.set_ylim(-15, 15)
# ax.set_zlim(-15, 15)
plt.show()

l0[-1]/l0[0]

In [ ]:
vertices_temp = np.copy(net.vertices)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation

# Compute rotation to align points 1, 3, and 6 horizontally
vertices = np.copy(vertices_temp)
# Define rotation matrix around x-axis
def rotate_x(theta, points):
    R = np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]
    ])
    return np.dot(points, R.T)

# Apply rotation
theta = np.pi/6  # Replace with your desired angle in radians
vertices = rotate_x(theta, vertices)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
ax.set_axis_off()
ax.grid(False)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])
lims = 30
ax.set_xlim(-lims, lims)
ax.set_ylim(-lims, lims)
ax.set_zlim(-lims, lims)

# Plot vertices
for edge in net.edges[cable_slice]:
    v1, v2 = vertices[edge[0]], vertices[edge[1]]
    ax.plot([v1[0], v2[0]], [v1[1], v2[1]], [v1[2], v2[2]], 'b-')
for edge in edges[strut_slice]:
    v1, v2 = vertices[edge[0]], vertices[edge[1]]
    ax.plot([v1[0], v2[0]], [v1[1], v2[1]], [v1[2], v2[2]], 'r-')

# Animation function
def update(frame):
    ax.view_init(elev=15, azim=frame)
    return fig,

ani = animation.FuncAnimation(fig, update, frames=np.linspace(0, 360, num=180), interval=100)

plt.show()
writer = animation.FFMpegWriter(fps=15)
ani.save(os.path.join(BYU_UW_root, 'images', 'tensegrity', f'rotation_{model_name}.mp4'), writer=writer)

In [ ]:
frames = [0, 45, 90]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
ax.set_axis_off()
ax.grid(False)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_zticklabels([])
plt.tight_layout()
lims = 30
ax.set_xlim(-lims, lims)
ax.set_ylim(-lims, lims)
ax.set_zlim(-lims, lims)

# Plot vertices
for edge in net.edges[cable_slice]:
    v1, v2 = vertices[edge[0]], vertices[edge[1]]
    ax.plot([v1[0], v2[0]], [v1[1], v2[1]], [v1[2], v2[2]], 'b-')
for edge in edges[strut_slice]:
    v1, v2 = vertices[edge[0]], vertices[edge[1]]
    ax.plot([v1[0], v2[0]], [v1[1], v2[1]], [v1[2], v2[2]], 'r-')
for i, v in enumerate(vertices):
    # White label (larger font) as a background
    ax.text(v[0], v[1], v[2], str(i + 1), color='white', fontsize=22, ha='center', va='center')
    # Black label (smaller font) on top for contrast
    ax.text(v[0] + 0.5, v[1] + 0.5, v[2] + 0.5, str(i + 1), color='black', fontsize=18, ha='center', va='center')

for frame in frames:
    ax.view_init(elev=15, azim=frame)
    plt.show()
    fig.savefig(os.path.join(BYU_UW_root, 'images', 'tensegrity', f'rotation_{model_name}_{frame}.png'))